# MNIST in PyTorch

In [1]:
import os

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [3]:
import brevitas.nn as qnn
from brevitas.core.quant import QuantType

***

## Define model etc.

In [20]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        config = {'weight_quant_type': QuantType.INT, 'weight_bit_width': 8}
        self.conv1 = qnn.QuantConv2d(1, 10, kernel_size=5, **config)
        self.conv2 = qnn.QuantConv2d(10, 20, kernel_size=5, **config)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = qnn.QuantLinear(320, 50, bias=False, **config)
        self.fc2 = qnn.QuantLinear(50, 10, bias=False, **config)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [8]:
def train(log_interval, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [9]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, size_average=False).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

***

## Settings

In [10]:
batch_size=64
test_batch_size=128
data_dir = os.path.join(os.getcwd(), 'data')
lr = 0.1
momentum = 0.5
epochs = 10

In [12]:
use_cuda = torch.cuda.is_available()
print('Using GPU...' if use_cuda else 'NOT using GPU...')

device = torch.device('cuda:0' if use_cuda else 'cpu')

Using GPU...


In [13]:
seed = 1
torch.manual_seed(seed)

***

## Get dataset

In [ ]:
# Since torchvision v0.2.1
datasets.MNIST(data_dir, download=True)

In [15]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(
        data_dir, train=True, download=True,
        transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])),
    batch_size=batch_size, shuffle=True, num_workers=2)

In [16]:
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(
        data_dir, train=False, 
        transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])),
    batch_size=test_batch_size, shuffle=True, num_workers=2)

***

## Start training

In [21]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [22]:
for epoch in range(0, epochs):
    train(500, model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.326479
Train Epoch: 0 [32000/60000 (53%)]	Loss: 0.483524


D:\Program\miniconda3\envs\pytorch\lib\site-packages\torch\nn\_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Test set: Average loss: 0.1174, Accuracy: 9645/10000 (96%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.124605
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.220145
Test set: Average loss: 0.0731, Accuracy: 9768/10000 (98%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.130023
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.506727
Test set: Average loss: 0.0656, Accuracy: 9796/10000 (98%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.227612
Train Epoch: 3 [32000/60000 (53%)]	Loss: 0.204421
Test set: Average loss: 0.0888, Accuracy: 9725/10000 (97%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.122698
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.214428
Test set: Average loss: 0.0554, Accuracy: 9815/10000 (98%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 0.118839
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.255303
Test set: Average loss: 0.0573, Accuracy: 9824/10000 (98%)

Train Epoch: 6 [0/60000 (0%)]	Loss: 0.127505
Train Epoch: 6 [32000/60000 (53%)]	Loss: 0.219499
Test set: Average loss: 0.0600, Accuracy: 9825/10000 (98%)

Train Epoc

***

## Save trained model

In [23]:
torch.save(model.state_dict(), 'mnist_quant.pt')